In [1]:
using MLDatasets
using Statistics
using PyPlot
using Distributions
using ProgressMeter
using Random
using MLBase
using Optim

In [2]:
# load full training set
train_x, train_y = MNIST.traindata()

# load full test set
test_x,  test_y  = MNIST.testdata()

xtrain0 = zeros(784,12000)
xtrain1 = zeros(784,12000)
xlabel =zeros(12000)
global counter0=0
global counter1=0
print(size(train_x,3))
for i=1:size(train_x,3)
    if train_y[i]==7
        global counter0=counter0+1
        xtrain0[:,counter0]=vec(1.0*train_x[:,:,i])
    end
    if train_y[i]==9
        global counter1=counter1+1
        xtrain1[:,counter1]=vec(1.0*train_x[:,:,i])
    end            
end
print(counter0,"  ",counter1)
xtest0 = zeros(784,12000)
xtest1 = zeros(784,12000)
xtestlabel =zeros(12000)
global counter0=0
global counter1=0
for i=1:size(test_x,3)
    if test_y[i]==7
        global counter0=counter0+1
        xtest0[:,counter0]=vec(1.0*test_x[:,:,i])
    end
    if test_y[i]==9
        global counter1=counter1+1
        xtest1[:,counter1]=vec(1.0*test_x[:,:,i])
    end            
end
print("  ",counter0,"  ",counter1)

600006265  5949  1028  1009

In [3]:
Ntrain=5900*2; Ntest=1000*2

x=[xtrain0[:,1:Int(Ntrain/2)] xtrain1[:,1:Int(Ntrain/2)]]
c=ones(Int,Ntrain)
for i=1:Int(Ntrain/2)
    c[i]=0
end

xtest=[xtest0[:,1:Int(Ntest/2)] xtest1[:,1:Int(Ntest/2)]]
ctest=ones(Int,Ntest)
for i=1:Int(Ntest/2)
    ctest[i]=0
end

In [4]:
function plot_image(x)
    xx=reshape(x,28,28)
    xxx=zeros(28,28)
    for i=1:28
        for j=1:28
            xxx[29-i,j]=xx[j,i]
        end
    end
    pcolormesh(xxx,cmap="gray")
    axis("equal")
    gca()[:set_axis_off]()
end

plot_image (generic function with 1 method)

In [5]:
function loglik(theta,c,x)
    L=0.0
    for n=1:size(x,2)
        L = L+log(sigmoid( (2*c[n]-1).*sum(theta.*x[:,n])))    
    end
    return L
end

loglik (generic function with 1 method)

In [6]:
function grad_theta(theta,c,x)
    g=zeros(size(x))
    cbar=2*c-1
    g = cbar*(1-sigmoid(cbar.*sum(theta.*x))).*x
end

grad_theta (generic function with 1 method)

In [7]:
sigmoid(x) = 1.0 ./ (1.0 .+exp.(-x))

function logistic_regression(Nloops,lr,lambda,x_train,c_train)
    theta=zeros(784)
    learning_rate=lr
    LL=zeros(Nloops)
    train_size=length(c_train)
    prev_loss=1e6
    
    for loop=1:Nloops 
        grad=zeros(784)
        for n=1:train_size
            grad=grad + grad_theta(theta,c_train[n],x_train[:,n])
        end
        theta = theta+learning_rate*(grad/train_size-lambda*theta)
        if loop%100==0
            llk = loglik(theta,c_train,x_train)
            L2norm_sqrd = sum(theta.^2)
            loss = -llk+0.5*lambda*L2norm_sqrd
            println("loop= $(loop) llk = $(round(llk; digits=2)) L2norm^2 = $(round(L2norm_sqrd; digits=4)) loss = $(round(loss; digits=2))")
            if (prev_loss-loss)<3 && loop>199#stopping criteria
                break
            else
                prev_loss=loss
            end
        end
    end
    return theta
end

logistic_regression (generic function with 1 method)

In [8]:
function generate_unit_vector(dim)
    v = randn(dim,1)
    norm = sqrt(sum(v.^2))
    v/norm
end

generate_unit_vector (generic function with 1 method)

In [9]:
function logistic_regression_dp(Nloops,lr,lambda,epsilon,x_train,c_train)
    theta=zeros(784)
    learning_rate=lr
    LL=zeros(Nloops)
    train_size=length(c_train)
    prev_loss=1e6
    b = generate_unit_vector(784)*rand(Gamma(784,2*28/epsilon),1)
    for loop=1:Nloops 
        grad=zeros(784)
        for n=1:train_size
            grad=grad + grad_theta(theta,c_train[n],x_train[:,n])
        end
        theta = theta+learning_rate*((grad-b)/train_size-lambda*theta)
        if loop%10==0
            llk = loglik(theta,c_train,x_train)
            loss = -llk+0.5*lambda*sum(theta.^2)
            L2norm = sqrt(sum(theta.^2))
            if (prev_loss-loss)<0.3 && loop>19#stopping criteria
                println("    loop= $(loop) llk = $(round(llk; digits=2)) L2norm = $(round(L2norm; digits=3)) loss = $(round(loss; digits=2))")
                break
            else
                prev_loss=loss
            end
        end
    end
    return theta
end

logistic_regression_dp (generic function with 1 method)

In [13]:
#algorithm 2 

fold=10
Nloop=3000
Nnoise=16

# for lambda in (1000,500,200,100,50,20,10,5,2,1,0.5,0.2,0.1,0.05,0.02,1e-2,1e-3,1e-4,1e-5)

for lambda in (0.1)
    if lambda > 0.2
        lr=min(0.15,0.5/lambda)
    else
        lr=0.3
    end
    
    valid_noise_acc,test_noise_acc=Dict(),Dict()
    
    for epsilon in (3,1,0.8,0.5,0.3,0.1)#reduced
        valid_noise_acc[epsilon],test_noise_acc[epsilon]=zeros(Nnoise,fold),zeros(Nnoise,fold)
    end
    
    for epsilon in (3,1,0.8,0.5,0.3,0.1)#reduced
        for k=1:Nnoise
            train_index=collect(Kfold(5900*2,10))

            for i=1:fold
#                 println("lambda = $(lambda) e =$(epsilon) run = $(k) fold = $(i) lr = $(lr)")
                train_size = length(train_index[i])
                valid_size = 5900*2-train_size
                valid_index = train_index[i]
                x_train = zeros(784,train_size)
                x_valid = zeros(784,valid_size)
                for j=1:train_size
                    x_train[:,j] = x[:,train_index[i][j]]
                end

                c_train = [c[x] for x in train_index[i]]
                valid_index = filter!(x->x∉train_index[i],[x for x=1:5900*2])
                for j=1:valid_size
                    x_valid[:,j] = x[:,valid_index[j]]
                end        

                c_valid = [c[x] for x in valid_index]

                theta = logistic_regression_dp(Nloop,lr,lambda,epsilon,x_train,c_train)
                
                #     check validation data accuracy
                cvalidpred=zeros(Int,valid_size)
                for n=1:valid_size
                    if sigmoid(sum(theta.*x_valid[:,n])) >0.5
                        cvalidpred[n]=1
                    end    
                end
                valid_noise_acc[epsilon][k,i] = mean(cvalidpred.==c_valid)

                ctestpred=zeros(Int,Ntest)
                for n=1:Ntest
                    if sigmoid(sum(theta.*xtest[:,n])) >0.5
                        ctestpred[n]=1
                    end    
                end
                test_noise_acc[epsilon][k,i] = mean(ctestpred.==ctest)   
                println("    valid acc = $(round(valid_noise_acc[epsilon][k,i]; digits=3))  test acc = $(round(test_noise_acc[epsilon][k,i]; digits=3))")  

            end
        end
    end
    println("====================================================")
    println("Summary: lambda = $(lambda) lr = $(lr)") 
    for epsilon in (3,1,0.8,0.5,0.3,0.1)#reduced
        println("e = $(epsilon) val_acc = $(round(mean(valid_noise_acc[epsilon]); digits=3)) test_acc = $(round(mean(test_noise_acc[epsilon]); digits=3))") 
    end
    println("====================================================")
end

    loop= 30 llk = -3429.35 L2norm = 7.974 loss = 3432.53
    valid acc = 0.868  test acc = 0.886
    loop= 30 llk = -3342.75 L2norm = 8.548 loss = 3346.4
    valid acc = 0.867  test acc = 0.878
    loop= 30 llk = -3322.58 L2norm = 8.165 loss = 3325.91
    valid acc = 0.862  test acc = 0.878
    loop= 30 llk = -3102.83 L2norm = 8.44 loss = 3106.39
    valid acc = 0.897  test acc = 0.909
    loop= 30 llk = -3266.07 L2norm = 8.468 loss = 3269.66
    valid acc = 0.856  test acc = 0.888
    loop= 30 llk = -3394.29 L2norm = 8.118 loss = 3397.59
    valid acc = 0.871  test acc = 0.866
    loop= 30 llk = -3461.05 L2norm = 8.272 loss = 3464.47
    valid acc = 0.858  test acc = 0.87
    loop= 30 llk = -3154.39 L2norm = 8.109 loss = 3157.68
    valid acc = 0.893  test acc = 0.897
    loop= 30 llk = -3299.79 L2norm = 8.593 loss = 3303.48
    valid acc = 0.881  test acc = 0.873
    loop= 30 llk = -3875.44 L2norm = 8.517 loss = 3879.07
    valid acc = 0.836  test acc = 0.854
    loop= 30 llk = -335

    loop= 30 llk = -3341.89 L2norm = 8.191 loss = 3345.24
    valid acc = 0.875  test acc = 0.878
    loop= 30 llk = -3202.05 L2norm = 7.942 loss = 3205.2
    valid acc = 0.887  test acc = 0.892
    loop= 30 llk = -3487.26 L2norm = 8.196 loss = 3490.62
    valid acc = 0.852  test acc = 0.873
    loop= 30 llk = -3430.43 L2norm = 7.992 loss = 3433.63
    valid acc = 0.885  test acc = 0.882
    loop= 30 llk = -3351.27 L2norm = 8.154 loss = 3354.59
    valid acc = 0.875  test acc = 0.888
    loop= 30 llk = -3319.02 L2norm = 8.549 loss = 3322.67
    valid acc = 0.869  test acc = 0.899
    loop= 30 llk = -3855.56 L2norm = 8.149 loss = 3858.88
    valid acc = 0.857  test acc = 0.863
    loop= 30 llk = -3484.47 L2norm = 8.169 loss = 3487.81
    valid acc = 0.858  test acc = 0.874
    loop= 30 llk = -3039.84 L2norm = 8.446 loss = 3043.41
    valid acc = 0.885  test acc = 0.888
    loop= 30 llk = -3245.47 L2norm = 8.3 loss = 3248.91
    valid acc = 0.897  test acc = 0.884
    loop= 30 llk = -368

    loop= 20 llk = -6605.35 L2norm = 18.832 loss = 6623.08
    valid acc = 0.763  test acc = 0.776
    loop= 20 llk = -8848.46 L2norm = 19.675 loss = 8867.81
    valid acc = 0.723  test acc = 0.734
    loop= 20 llk = -5564.84 L2norm = 18.315 loss = 5581.62
    valid acc = 0.803  test acc = 0.786
    loop= 20 llk = -5690.93 L2norm = 18.317 loss = 5707.7
    valid acc = 0.781  test acc = 0.8
    loop= 20 llk = -6158.41 L2norm = 18.864 loss = 6176.2
    valid acc = 0.803  test acc = 0.814
    loop= 20 llk = -8625.51 L2norm = 20.116 loss = 8645.75
    valid acc = 0.704  test acc = 0.7
    loop= 20 llk = -8867.45 L2norm = 20.113 loss = 8887.67
    valid acc = 0.719  test acc = 0.703
    loop= 20 llk = -6639.53 L2norm = 18.686 loss = 6656.99
    valid acc = 0.789  test acc = 0.78
    loop= 20 llk = -6098.7 L2norm = 18.897 loss = 6116.55
    valid acc = 0.779  test acc = 0.812
    loop= 20 llk = -5609.44 L2norm = 18.26 loss = 5626.11
    valid acc = 0.82  test acc = 0.829
    loop= 20 llk = -

    loop= 20 llk = -5305.98 L2norm = 17.682 loss = 5321.61
    valid acc = 0.826  test acc = 0.842
    loop= 20 llk = -6134.88 L2norm = 19.37 loss = 6153.64
    valid acc = 0.798  test acc = 0.799
    loop= 20 llk = -6105.55 L2norm = 17.812 loss = 6121.41
    valid acc = 0.776  test acc = 0.792
    loop= 20 llk = -9232.02 L2norm = 18.789 loss = 9249.67
    valid acc = 0.711  test acc = 0.708
    loop= 20 llk = -8118.79 L2norm = 20.134 loss = 8139.06
    valid acc = 0.731  test acc = 0.745
    loop= 20 llk = -6593.26 L2norm = 19.31 loss = 6611.91
    valid acc = 0.745  test acc = 0.767
    loop= 20 llk = -6991.47 L2norm = 17.719 loss = 7007.16
    valid acc = 0.769  test acc = 0.781
    loop= 20 llk = -6197.76 L2norm = 18.637 loss = 6215.13
    valid acc = 0.804  test acc = 0.803
    loop= 20 llk = -6173.73 L2norm = 18.046 loss = 6190.01
    valid acc = 0.774  test acc = 0.774
    loop= 20 llk = -7253.1 L2norm = 19.296 loss = 7271.72
    valid acc = 0.75  test acc = 0.764
    loop= 20 l

    loop= 20 llk = -6152.56 L2norm = 23.385 loss = 6179.9
    valid acc = 0.825  test acc = 0.837
    loop= 20 llk = -9482.56 L2norm = 24.29 loss = 9512.06
    valid acc = 0.735  test acc = 0.73
    loop= 20 llk = -8043.92 L2norm = 24.323 loss = 8073.5
    valid acc = 0.788  test acc = 0.794
    loop= 20 llk = -7076.87 L2norm = 22.494 loss = 7102.17
    valid acc = 0.781  test acc = 0.789
    loop= 20 llk = -10457.11 L2norm = 22.979 loss = 10483.51
    valid acc = 0.669  test acc = 0.662
    loop= 20 llk = -12223.31 L2norm = 23.887 loss = 12251.84
    valid acc = 0.657  test acc = 0.684
    loop= 20 llk = -9788.8 L2norm = 23.452 loss = 9816.3
    valid acc = 0.717  test acc = 0.732
    loop= 20 llk = -12356.42 L2norm = 24.349 loss = 12386.07
    valid acc = 0.612  test acc = 0.65
    loop= 20 llk = -11885.2 L2norm = 22.407 loss = 11910.3
    valid acc = 0.602  test acc = 0.618
    loop= 20 llk = -6845.02 L2norm = 23.585 loss = 6872.84
    valid acc = 0.794  test acc = 0.809
    loop= 2

...

    loop= 20 llk = -18130.01 L2norm = 36.336 loss = 18196.02
    valid acc = 0.614  test acc = 0.627
    loop= 20 llk = -13608.25 L2norm = 38.489 loss = 13682.32
    valid acc = 0.692  test acc = 0.712
    loop= 20 llk = -16996.12 L2norm = 38.321 loss = 17069.55
    valid acc = 0.669  test acc = 0.676
    loop= 20 llk = -9680.47 L2norm = 37.586 loss = 9751.11
    valid acc = 0.763  test acc = 0.804
    loop= 20 llk = -20720.88 L2norm = 37.3 loss = 20790.45
    valid acc = 0.563  test acc = 0.592
    loop= 20 llk = -14951.83 L2norm = 35.975 loss = 15016.54
    valid acc = 0.677  test acc = 0.721
    loop= 20 llk = -20740.36 L2norm = 39.157 loss = 20817.02
    valid acc = 0.646  test acc = 0.656
    loop= 20 llk = -17745.52 L2norm = 38.305 loss = 17818.89
    valid acc = 0.608  test acc = 0.591
    loop= 20 llk = -15649.4 L2norm = 38.338 loss = 15722.89
    valid acc = 0.666  test acc = 0.679
    loop= 20 llk = -13794.05 L2norm = 34.39 loss = 13853.18
    valid acc = 0.663  test acc = 0.

    valid acc = 0.564  test acc = 0.547
    loop= 20 llk = -25296.65 L2norm = 68.536 loss = 25531.5
    valid acc = 0.742  test acc = 0.738
    loop= 20 llk = -22233.35 L2norm = 59.213 loss = 22408.66
    valid acc = 0.741  test acc = 0.746
    loop= 20 llk = -31577.47 L2norm = 56.691 loss = 31738.16
    valid acc = 0.658  test acc = 0.63
    loop= 20 llk = -26648.88 L2norm = 60.705 loss = 26833.13
    valid acc = 0.65  test acc = 0.666
    loop= 20 llk = -30581.22 L2norm = 64.127 loss = 30786.83
    valid acc = 0.647  test acc = 0.654
    loop= 20 llk = -31628.89 L2norm = 61.641 loss = 31818.87
    valid acc = 0.62  test acc = 0.65
    loop= 20 llk = -26450.8 L2norm = 57.629 loss = 26616.86
    valid acc = 0.657  test acc = 0.648
    loop= 20 llk = -54718.84 L2norm = 62.899 loss = 54916.66
    valid acc = 0.419  test acc = 0.413
    loop= 20 llk = -40718.29 L2norm = 64.603 loss = 40926.97
    valid acc = 0.568  test acc = 0.564
    loop= 20 llk = -26265.42 L2norm = 62.886 loss = 26463

    valid acc = 0.594  test acc = 0.586
    loop= 20 llk = -36456.22 L2norm = 65.05 loss = 36667.79
    valid acc = 0.572  test acc = 0.586
    loop= 20 llk = -57257.69 L2norm = 64.618 loss = 57466.46
    valid acc = 0.554  test acc = 0.558
    loop= 20 llk = -34716.67 L2norm = 63.216 loss = 34916.49
    valid acc = 0.605  test acc = 0.627
    loop= 20 llk = -28335.71 L2norm = 63.514 loss = 28537.41
    valid acc = 0.687  test acc = 0.718
    loop= 20 llk = -29974.36 L2norm = 61.472 loss = 30163.3
    valid acc = 0.66  test acc = 0.648
    loop= 20 llk = -29594.61 L2norm = 62.016 loss = 29786.92
    valid acc = 0.608  test acc = 0.594
    loop= 20 llk = -30272.17 L2norm = 63.534 loss = 30474.0
    valid acc = 0.636  test acc = 0.646
    loop= 20 llk = -38677.86 L2norm = 59.739 loss = 38856.3
    valid acc = 0.569  test acc = 0.58
    loop= 20 llk = -31506.1 L2norm = 63.496 loss = 31707.68
    valid acc = 0.621  test acc = 0.632
    loop= 20 llk = -37809.66 L2norm = 61.812 loss = 38000.

    valid acc = 0.637  test acc = 0.65
    loop= 20 llk = -175972.5 L2norm = 185.546 loss = 177693.87
    valid acc = 0.489  test acc = 0.48
    loop= 20 llk = -177979.35 L2norm = 188.32 loss = 179752.57
    valid acc = 0.562  test acc = 0.542
    loop= 20 llk = -128602.19 L2norm = 182.281 loss = 130263.5
    valid acc = 0.642  test acc = 0.654
    loop= 20 llk = -112018.92 L2norm = 185.291 loss = 113735.55
    valid acc = 0.624  test acc = 0.597
    loop= 20 llk = -200769.5 L2norm = 193.637 loss = 202644.26
    valid acc = 0.484  test acc = 0.478
    loop= 20 llk = -142806.04 L2norm = 176.764 loss = 144368.33
    valid acc = 0.527  test acc = 0.518
    loop= 20 llk = -203040.61 L2norm = 191.783 loss = 204879.65
    valid acc = 0.505  test acc = 0.464
    loop= 20 llk = -99503.67 L2norm = 199.403 loss = 101491.75
    valid acc = 0.597  test acc = 0.646
    loop= 20 llk = -200660.57 L2norm = 185.456 loss = 202380.27
    valid acc = 0.514  test acc = 0.499
    loop= 20 llk = -152629.67 L

    loop= 20 llk = -117466.82 L2norm = 178.478 loss = 119059.53
    valid acc = 0.624  test acc = 0.657
    loop= 20 llk = -225305.06 L2norm = 176.181 loss = 226857.05
    valid acc = 0.463  test acc = 0.47
    loop= 20 llk = -124967.81 L2norm = 177.004 loss = 126534.34
    valid acc = 0.544  test acc = 0.52
    loop= 20 llk = -147247.57 L2norm = 189.686 loss = 149046.62
    valid acc = 0.585  test acc = 0.618
    loop= 20 llk = -223144.08 L2norm = 188.925 loss = 224928.71
    valid acc = 0.461  test acc = 0.462
    loop= 20 llk = -214770.46 L2norm = 181.855 loss = 216424.02
    valid acc = 0.338  test acc = 0.356
    loop= 20 llk = -146808.81 L2norm = 186.134 loss = 148541.11
    valid acc = 0.553  test acc = 0.567
    loop= 20 llk = -102738.98 L2norm = 190.189 loss = 104547.57
    valid acc = 0.579  test acc = 0.579
    loop= 20 llk = -154731.71 L2norm = 190.52 loss = 156546.6
    valid acc = 0.527  test acc = 0.56
    loop= 20 llk = -102953.34 L2norm = 181.067 loss = 104592.6
    va

In [11]:
#algorithm 1

fold=10
Nloop=4000
Nnoise=100
thetas = Dict()
# for lambda in (1000,500,200,100,50,20,10,5,2,1,0.5,0.2,0.1,0.05,0.02,1e-2,1e-3,1e-4,1e-5)
for lambda in (0.5)
    thetas[lambda]=Dict()
    if lambda > 0.2
        lr=min(0.15,0.5/lambda)
    else
        lr=0.3
    end
    train_index=collect(Kfold(5900*2,10))
    train_acc,valid_acc,test_acc=zeros(fold),zeros(fold),zeros(fold)
    
    valid_noise_acc,test_noise_acc=Dict(),Dict()
    
    for epsilon in (1e5,3e4,1e4,3e3,1e3,300,100,30,10,3,1,0.3,0.1)    
#     for epsilon in (1e5,3e4,1e4,3e3,1e3,800,500,300,100,80,50,30,10,8,5,3,1,0.8,0.5,0.3,0.1)
        
#             10, 8, 5, 3, 1, 0.8, 0.5, 0.3, 0.1, 
#                     0.08, 0.05, 0.03, 0.01, 0.009, 0.008, 0.007, 0.006, 0.005, 
#                     0.004, 0.003, 0.002, 0.001, 0.0008, 0.0005, 0.0003, 0.0001)
        valid_noise_acc[epsilon],test_noise_acc[epsilon]=zeros(fold,Nnoise),zeros(fold,Nnoise)
    end
    
    for i=1:fold
        println("fold = $(i) lambda = $(lambda) lr = $(lr)")
        train_size = length(train_index[i])
        valid_size = 5900*2-train_size
        valid_index = train_index[i]
        x_train = zeros(784,train_size)
        x_valid = zeros(784,valid_size)
        for j=1:train_size
            x_train[:,j] = x[:,train_index[i][j]]
        end
        
        c_train = [c[x] for x in train_index[i]]
        valid_index = filter!(x->x∉train_index[i],[x for x=1:5900*2])
        for j=1:valid_size
            x_valid[:,j] = x[:,valid_index[j]]
        end        
        
        c_valid = [c[x] for x in valid_index]
        
        theta = logistic_regression(Nloop,lr,lambda,x_train,c_train)
        thetas[lambda][i]=theta
        #     check training data accuracy
        ctrainpred=zeros(Int,train_size)
        for n=1:train_size
            if sigmoid(sum(theta.*x_train[:,n])) >0.5
                ctrainpred[n]=1
            end    
        end
        train_acc[i] = mean(ctrainpred.==c_train)
        println("train accuracy = $(round(train_acc[i]; digits=3))")
        
        #     check validation data accuracy
        cvalidpred=zeros(Int,valid_size)
        for n=1:valid_size
            if sigmoid(sum(theta.*x_valid[:,n])) >0.5
                cvalidpred[n]=1
            end    
        end
        valid_acc[i] = mean(cvalidpred.==c_valid)
        println("validation accuracy = $(round(valid_acc[i]; digits=3))")

        ctestpred=zeros(Int,Ntest)
        for n=1:Ntest
            if sigmoid(sum(theta.*xtest[:,n])) >0.5
                ctestpred[n]=1
            end    
        end
        test_acc[i] = mean(ctestpred.==ctest)   
        println("test accuracy = $(round(test_acc[i]; digits=3))")  
        
        for epsilon in (1e5,3e4,1e4,3e3,1e3,300,100,30,10,3,1,0.3,0.1)  
            for j=1:Nnoise 
                noise = generate_unit_vector(784)*rand(Gamma(784,2*28/(epsilon*train_size*lambda)),1)
                theta_noise = theta + noise
                
                cvalidpred=zeros(Int,valid_size)
                for n=1:valid_size
                    if sigmoid(sum(theta_noise.*x_valid[:,n])) >0.5
                        cvalidpred[n]=1
                    end    
                end
                valid_noise_acc[epsilon][i,j] = mean(cvalidpred.==c_valid)

                ctestpred=zeros(Int,Ntest)
                for n=1:Ntest
                    if sigmoid(sum(theta_noise.*xtest[:,n])) >0.5
                        ctestpred[n]=1
                    end    
                end
                test_noise_acc[epsilon][i,j] = mean(ctestpred.==ctest)
            end
        end
    end
    println("====================================================")
    println("Summary: lambda = $(lambda) lr = $(lr)") 
    println("train accuracy = $(round(mean(train_acc); digits=3))") 
    println("validation accuracy = $(round(mean(valid_acc); digits=3))") 
    println("test accuracy = $(round(mean(test_acc); digits=3))")
    for epsilon in (1e5,3e4,1e4,3e3,1e3,300,100,30,10,3,1,0.3,0.1)  
        println("e = $(epsilon) val_acc = $(round(mean(valid_noise_acc[epsilon]); digits=3)) test_acc = $(round(mean(test_noise_acc[epsilon]); digits=3))") 
    end
    println("====================================================")
end

#write essay


fold = 1 lambda = 0.5 lr = 0.15
loop= 100 llk = -3952.62 L2norm^2 = 0.4001 loss = 3952.72
loop= 200 llk = -3952.6 L2norm^2 = 0.4001 loss = 3952.7
train accuracy = 0.93
validation accuracy = 0.936
test accuracy = 0.932
fold = 2 lambda = 0.5 lr = 0.15
loop= 100 llk = -3948.78 L2norm^2 = 0.4 loss = 3948.88
loop= 200 llk = -3948.76 L2norm^2 = 0.4 loss = 3948.86
train accuracy = 0.93
validation accuracy = 0.931
test accuracy = 0.93
fold = 3 lambda = 0.5 lr = 0.15
loop= 100 llk = -3945.14 L2norm^2 = 0.4005 loss = 3945.24
loop= 200 llk = -3945.13 L2norm^2 = 0.4005 loss = 3945.23
train accuracy = 0.932
validation accuracy = 0.918
test accuracy = 0.932
fold = 4 lambda = 0.5 lr = 0.15
loop= 100 llk = -3954.7 L2norm^2 = 0.4 loss = 3954.8
loop= 200 llk = -3954.69 L2norm^2 = 0.4 loss = 3954.79
train accuracy = 0.93
validation accuracy = 0.931
test accuracy = 0.93
fold = 5 lambda = 0.5 lr = 0.15
loop= 100 llk = -3955.21 L2norm^2 = 0.4 loss = 3955.31
loop= 200 llk = -3955.2 L2norm^2 = 0.4 loss = 3955

In [15]:
for lambda in (1000,500,200,100,50,20,10,5,2,1,0.5,0.2,0.1,0.05,0.02,1e-2,1e-3,1e-4,1e-5)
    dummy=0
    for i=1:10
        dummy+=sum(thetas[lambda][i].^2)
    end
    L2norm=sqrt((dummy/10))
    println("lambda $(lambda)   L2norm $(L2norm)")
end


lambda 1000   L2norm 0.000745565851141742
lambda 500   L2norm 0.0014898472620929395
lambda 200   L2norm 0.0037140632413393358
lambda 100   L2norm 0.007394002281038332
lambda 50   L2norm 0.014656058446798629
lambda 20   L2norm 0.03570878061722349
lambda 10   L2norm 0.0685993644594753
lambda 5   L2norm 0.12745082342156133
lambda 2   L2norm 0.26559026846012984
lambda 1   L2norm 0.42533544878548424
lambda 0.5   L2norm 0.8710902108076706
lambda 0.2   L2norm 0.9740658302646108
lambda 0.1   L2norm 1.2841630760282836
lambda 0.05   L2norm 1.643522489537571
lambda 0.02   L2norm 2.2078270049403113
lambda 0.01   L2norm 2.7210494794056164
lambda 0.001   L2norm 5.016195441287009
lambda 0.0001   L2norm 6.653024243074171
lambda 1.0e-5   L2norm 6.943866405461439
